### **TOOLS !!**

In [17]:
import os 
import requests 
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
from ollama import Client
from groq import Groq
import gradio as gr
import json

In [2]:
load_dotenv(override=True)

python-dotenv could not parse statement starting at line 11


True

In [3]:
openai = OpenAI()
MODEL_GPT = 'gpt-4.1-nano'

client = Client()
MODEL_LLAMA = 'llama3.1:8b'

In [4]:
system_message = "You are a helpful assistant for an Airline called FlightAI."
system_message += "Give shoert, courteous answers, no more than one sentence."
system_message += "Always be accurate. If you dont know the answer, say so."

In [7]:
def chat(message, history):
    messages = [{'role': 'system', 'content': system_message}]
    for user_message, assistant_message in history:
        messages.append({'role': 'user', 'content': user_message})
        messages.append({'role': 'assistant', 'content': assistant_message})
    messages.append({'role': 'user', 'content': message})

    response = openai.chat.completions.create(model=MODEL_GPT, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat).launch()

d:\LLM_Engineering\.venv\Lib\site-packages\gradio\chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


### **Tools**  
+ Tools are incredibly powerful feature provided by the frontier LLMs.
+ With tools, you can write a function, and have the LLM call that function as part of its response.
+ Sounds almost spooky.., we are giving it power to run code on our machine ??
+ Well, kinda...

In [8]:
# Let's start by making a useful function

ticket_prices = {'london': '$799', 'paris': '$899', 'tokyo': '$1400', 'berlin': '$499'}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, 'unknown')

In [10]:
# is it working..? Absolutely yes!!

get_ticket_price('Berlin')

Tool get_ticket_price called for Berlin


'$499'

> There's a particular dictionary structure that's required to describe our function:

In [ ]:
# the function

price_function = {
    "name": "get_ticket_price", 
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'how much does it take to this city?'", 
    "parameters": {
        "type": "object", 
        "properties": {
            "destination_city": {
                "type": "string", 
                "description": "The city that the customer wants to travel to.", 
            }, 
        }, 
        "required": ["destination_city"], 
        "additionalProperties": False
    }
}

In [13]:
# and this is included in a list of tools

tools = [{'type': 'function', 'function': price_function}]

### **Getting OpenAI to use our Tool**  
+ There's some fiddly stuff to allow OpenAI "to call our tool".
+ What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool. 
+ Here's how the new chat function looks like.


In [19]:
def chat(message, history):
    messages = [{'role': 'system', 'content': system_message}]
    for user, assistant in history:
        messages.append({'role': 'user', 'content': user})
        messages.append({'role': 'assistant', 'content': assistant})
    messages.append({'role': 'user', 'content': message})

    response = openai.chat.completions.create(model=MODEL_GPT, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL_GPT, messages=messages)

    return response.choices[0].message.content

In [18]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        'role': 'tool', 
        'content': json.dumps({'destination_city': city, 'price': price}), 
        'tool_call_id': message.tool_calls[0].id
    }
    return response, city

In [ ]:
gr.ChatInterface(fn=chat).launch()

d:\LLM_Engineering\.venv\Lib\site-packages\gradio\chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for London
Tool get_ticket_price called for Tokyo
Tool get_ticket_price called for Berlin
Tool get_ticket_price called for Timbuktu
